In [1]:
# FIND N-SITE POS. IN N-GLYCOSYLATION SEQUON 
# TOOLS

import numpy as np
import pandas as pd
import regex as re

def findNPosBySequon(seq, species=''):
    pattern = '(?=((?i)(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)))'

    npos = [m.start(0)+1 for m in re.finditer(pattern, seq, overlapped = True)]
    sequon = [seq[m.start(0):m.start(0)+3] for m in re.finditer(pattern, seq, overlapped = True)]


#     return result
#     f = open(f'{species}_NPosBySequon.txt', 'w') 
#     f.writelines(ind)
#     f.close()
    # Generate dataframe from result
    df = [(npos[i], sequon[i]) for i in range(len(npos))]
    df = pd.DataFrame(df)
    df.columns = ['N-site', 'Sequon']
    result = df.to_excel(f'{species}_findNPosBySequon.xlsx', header=True, index=False)
    

def alp_cutter(seq, species=''):
    # cutting sites for α-Lytic Protease
    T_ind = [pos for pos, char in enumerate(seq) if char == 'T' or char == 't'] 
    A_ind = [pos for pos, char in enumerate(seq) if char == 'A' or char == 'a']
    S_ind = [pos for pos, char in enumerate(seq) if char == 'S' or char == 's']
    V_ind = [pos for pos, char in enumerate(seq) if char == 'V' or char == 'v']

    a = set(T_ind).union(A_ind).union(S_ind).union(V_ind)
    a = list(a)
    a.sort() #cutting site result
    #all add one to cut the fragments.
    a = np.array(a)
    a += 1
    a = list(a)

    frag_lst = [] #store the cutted frag.
    first_frag = seq[0: a[0]]
    frag_lst.append(first_frag)
    for i in range(len(a)):
        if i != (len(a) -1): #last index recorded.
            frag = seq[a[i]:a[i +1]]
            frag_lst.append(frag)
        elif i == (len(a) -1):
            #last frag
            frag = seq[a[i]:]
            frag_lst.append(frag)

    frag_len = [len(j) for j in frag_lst]
    cleavage_site = list(np.cumsum(frag_len))

    # Generate dataframe from list and write to xlsx.
    frag_data = [(cleavage_site[i], frag_lst[i], frag_len[i]) for i in range(len(frag_lst))]
    frag_df = pd.DataFrame(frag_data)
    frag_df.columns = ['Position of cleavage site', 'Resulting peptide sequence', 'Peptide length [aa]']
    result = frag_df.to_excel(f'{species}_aLP_fragments.xlsx', header=True, index=False)

# main
species = ''
input_seq = ''
alp_cutter(input_seq, species=species)
findNPosBySequon(input_seq, species=species)